# Multiple Backend
`rubicon-ml` allows users to instantiate `Rubicon` objects with multiple backends to write to/read from. These backends include local, memory, and S3 repositories. Here's a walk through of how one might instantiate and use a `Rubicon` object with multiple backends.

In [1]:
from rubicon_ml import Rubicon

In [2]:
#rb = Rubicon(persistence="memory")
#or
#rb = Rubicon(persistence="filesystem")

However, when we want multiple backends we utilize the `composite_config` kwarg:

In [3]:
#example multiple backend instantiaiton
rb = Rubicon(composite_config=[
    {"persistence": "filesystem", "root_dir": "./local/rootA"},
    {"persistence": "filesystem", "root_dir": "./local/rootB"},
])

### Write Commands
The following commands write to all insantiated backend repositories:

In [4]:
project = rb.create_project("example_project")

experiment = project.log_experiment("example_experiment")

artifact = experiment.log_artifact(data_bytes=b"bytes", name="example_artifact")
import pandas as pd
dataframe = experiment.log_dataframe(pd.DataFrame([[5, 0, 0], [0, 5, 1], [0, 0, 4]], columns=["x", "y", "z"]))
feature = experiment.log_feature("year")
metric = experiment.log_metric("accuracy", .8)
parameter = experiment.log_parameter("n_estimators")

Let's verify both of our backends have been written to:

In [5]:
!ls "./local/rootA"
print("\n")
!ls "./local/rootA/exampleproject"
print("\n")
!ls "./local/rootA/exampleproject/experiments"
print("\n")
!ls "./local/rootA/exampleproject/experiments/{experiment.id}" 
print("\n")
!ls "./local/rootA/exampleproject/experiments/{experiment.id}/artifacts"
print("\n")
!ls "./local/rootA/exampleproject/experiments/{experiment.id}/dataframes"
print("\n")
!ls "./local/rootA/exampleproject/experiments/{experiment.id}/features"
print("\n")
!ls "./local/rootA/exampleproject/experiments/{experiment.id}/metrics"
print("\n")
!ls "./local/rootA/exampleproject/experiments/{experiment.id}/parameters"

exampleproject


experiments   metadata.json


3cd4155e-08e3-4ae0-9c63-3566334d9a5e


artifacts     features      metrics
dataframes    metadata.json parameters


af003e41-6278-412c-8511-ccfbc2813cf3


c241e0c6-ce22-49fd-9555-7e73ba16e63d


year


accuracy


nestimators


In [6]:
!ls "./local/rootB"
print("\n")
!ls "./local/rootB/exampleproject"
print("\n")
!ls "./local/rootB/exampleproject/experiments"
print("\n")
!ls "./local/rootB/exampleproject/experiments/{experiment.id}" 
print("\n")
!ls "./local/rootB/exampleproject/experiments/{experiment.id}/artifacts"
print("\n")
!ls "./local/rootB/exampleproject/experiments/{experiment.id}/dataframes"
print("\n")
!ls "./local/rootB/exampleproject/experiments/{experiment.id}/features"
print("\n")
!ls "./local/rootB/exampleproject/experiments/{experiment.id}/metrics"
print("\n")
!ls "./local/rootB/exampleproject/experiments/{experiment.id}/parameters"

exampleproject


experiments   metadata.json


3cd4155e-08e3-4ae0-9c63-3566334d9a5e


artifacts     features      metrics
dataframes    metadata.json parameters


af003e41-6278-412c-8511-ccfbc2813cf3


c241e0c6-ce22-49fd-9555-7e73ba16e63d


year


accuracy


nestimators


### Read Commands
Now that we've seen both of our backends have been written to, let's see the read commands. Read commands will iterate over all backend repositories and return from the first one they are able to read from. A `RubiconException` will be raised if none of the backend repositories can be read the requested item(s).

In [7]:
projects = rb.projects()
print("projects: " + str(projects))
print("\n")

experiments = project.experiments()
print("experiments: " + str(experiments))
print("\n")

artifacts = experiment.artifacts()
print("artifacts: " + str(artifacts))
print("\n")

dataframes = experiment.dataframes()
print("dataframes: " + str(dataframes))
print("\n")

features = experiment.features()
print("features: " + str(features))
print("\n")

metrics = experiment.metrics()
print("metrics: " + str(metrics))
print("\n")

parameters = experiment.parameters()
print("parameters: " + str(parameters))
print("\n")

projects: [<rubicon_ml.client.project.Project object at 0x13322d810>]


experiments: [<rubicon_ml.client.experiment.Experiment object at 0x13322f100>]


artifacts: [<rubicon_ml.client.artifact.Artifact object at 0x13322f730>]


dataframes: [<rubicon_ml.client.dataframe.Dataframe object at 0x13322f6a0>]


features: [<rubicon_ml.client.feature.Feature object at 0x1331ee200>]


metrics: [<rubicon_ml.client.metric.Metric object at 0x13322d120>]


parameters: [<rubicon_ml.client.parameter.Parameter object at 0x13327c1c0>]




#### Additional Read Commands
Along with the commands demonstrated above, all other "read" type rubicon commands work the same way in that they will iterate over backend repositories and return from the first one they are able to read from. These include commands which read a specific logged object like `get_project()`, `experiment()`, `artifact()`, `dataframe()`, `metric()`, and `parameter()`.